# Libraries

In [1]:
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import time

import scienceplots
plt.style.use(['science', 'notebook'])

from tensorflow import keras
from keras import models
from keras import layers

import h5py
from tqdm import tqdm

# Synthetic images

In [ ]:
class SyntheticImagesGen:

    def __init__(self, number_configs: int, training: list[str]=['all'], L=40):
        if training == ['all']:
            training = ['para', 'ferro', 'neel', 'stripe']
        else:
            training = [element.lower() for element in training]

        self.number_configs = number_configs
        self.training = training
        self.L = L

    def spin_gen(self, conf: str):
        '''
        Generates a spin configuration for the given configuration. 
        The resulting data is a tuple of the different configurations each one can have.
        For example, ferromagnetic configurations can be spin up ferromagnetic or spin down ferromagnetic.
        In this case, the tuple will contain two LxL matrices, one per each type.
        '''
        spin_conf = []
        if conf == 'ferro':
            spin_conf = [np.ones((self.L, self.L)).astype(int), 
                        -np.ones((self.L, self.L)).astype(int)]
        elif conf == 'neel':
            spin = np.fromfunction(lambda i, j: (-1)**(i + j + (conf == 4)), 
                                (self.L, self.L)).astype(int)
            spin_conf = [spin, -spin]
        elif conf == 'stripe':
            spin = np.fromfunction(lambda i, j: (-1)**j, (self.L, self.L)).astype(int)
            spin_conf = [spin, -spin, spin.T, -spin.T]
        elif conf == 'para':
            spin_conf = np.random.choice([-1, 1], size=(self.L, self.L)).astype(int)
        return spin_conf
    
    def Info(self):
        return print(f'Number of configurations: {self.number_configs}\nTraining: {self.training} \nL: {self.L}\n')

    def dataGenerator(self):
        ''' 
        Generates synthetic data given a number of configurations and the type of training we want to do.
        If 'all', then it will generate all possible configurations evenly distributed. There are 4 types of configurations.
        If the number of configurations is not divisible by 4, the remaining configurations will be generated in a paramagnetic manner.
        '''
        start_time = time.time()
        print("Generating synthetic data...")
        config_dict = {
            'para': 1,
            'ferro': 2,
            'neel': 2,
            'stripe': 4
        }

        labels_dict = {
            'para': 0,
            'ferro': 1,
            'neel': 2,
            'stripe': 3
        }

        selected_dict = {k: v for k, v in config_dict.items() if k in self.training}

        selected_labels = {k: v for k, v in labels_dict.items() if k in self.training}
        
        total_configs_per_selected = self.number_configs // len(selected_dict.values())
        remaining_configs = self.number_configs % len(selected_dict.values())

        train_images = []
        train_labels = []

        if 'para' in self.training:
            for _ in range(total_configs_per_selected):
                train_images.append(self.spin_gen('para'))
                train_labels.append(labels_dict['para'])
            del selected_dict['para'], selected_labels['para']
        
        for conf in selected_dict:
            total_conf = total_configs_per_selected
            extra_configs = total_conf % selected_dict[conf]    
                
            if extra_configs !=0:
                remaining_configs += extra_configs
            
            total_conf = total_conf // selected_dict[conf]

            for _ in range(total_conf):
                train_images.extend(self.spin_gen(conf))
                for _ in range(config_dict[conf]):
                    train_labels.append(labels_dict[conf])

        if 'para' in self.training:
            for _ in range(remaining_configs):
                train_images.append(self.spin_gen('para'))
                train_labels.append(labels_dict['para'])
        else:
            for i in range(remaining_configs):
                index = i % len(self.training)
                train_images.append(self.spin_gen(self.training[index])[0])
                train_labels.append(labels_dict[self.training[index]])

        temp = list(zip(train_images, train_labels))
        random.shuffle(temp)
        train_images, train_labels = zip(*temp)

        print("Done!")

        end_time = time.time()
        elapsed_time = end_time - start_time
        print("Elapsed time:", elapsed_time, "seconds")
        return np.array(train_images), np.array(train_labels)


In [ ]:
data = SyntheticImagesGen(10003, training=['ferro','neel','stripe'], L=40)

In [ ]:
train_images, train_labels = data.dataGenerator()

In [ ]:
data.Info()

In [ ]:
train_images.shape

# Simulated images

In [5]:
class loader_and_saver:
    def __init__(self, path):
        self.path = path
    
    
    def saver(self, data):
        name = input("Enter the name of the file: ")
        name += '.h5'
        with h5py.File(name, 'w') as f:
            for i, arr in enumerate(tqdm(data, desc="Saving images", unit="array")):
                f.create_dataset(f'array_{i}', data=arr, compression='gzip', compression_opts=9)
        print("Files saved!")


    def loader(self, file_name):
        name = file_name
        if name[:-3] !='.h5':
            name += '.h5'

        loaded_list = []
        with h5py.File(name, 'r') as f:
            for key in tqdm(sorted(f.keys(), key=lambda x: int(x.split('_')[1])), 
                            desc="Loading arrays", unit="array"):
                loaded_list.append(f[key][:])
        print("Files loaded!")
        return loaded_list


    def checker(self, original_list, loaded_list):
        data_is_equal = True
        for original, loaded in zip(original_list, loaded_list):
            if not np.array_equal(original, loaded):
                data_is_equal = False
                break
        if data_is_equal:
            print("The original data set and the loaded data set are identical.")
        else:
            print("The original data set and the loaded data set are NOT identical.")


    def simulatedImages(self, index: int):
        print('Loading simulated images...')

        densityIndices = ['055','06', '061', '062', '063', '064', '065', '07', '08', '09','1']

        loadingPath = os.path.join(self.path,'data',f'data_p{densityIndices[index]}')
        
        simImages = self.loader(loadingPath)
        
        temperature = np.arange(0.0, 5.02, 0.02).tolist()
        dens_format = densityIndices[index][:1]+'.'+densityIndices[index][1:]
        print(f'Data of density p = {dens_format} succesfully loaded.')
        
        return simImages, temperature

In [6]:
loading_data = loader_and_saver(os.getcwd())

In [7]:
sim_images, temperature = loading_data.simulatedImages(5)

Loading simulated images...


Loading arrays:   0%|          | 0/142 [00:00<?, ?array/s]

Loading arrays: 100%|██████████| 142/142 [00:02<00:00, 68.97array/s]

Files loaded!
Data of density p = 0.64 succesfully loaded.


# Neural networks